In [1]:
import queueing_tool as qt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Define times
- higher rate for fast arrival time lambdas
- lower rate for slow arrival time lambdas

In [2]:
slow_rate = lambda t: 1.5
slow_arr = lambda t: qt.poisson_random_measure(t, slow_rate, 0.5)

fast_rate = lambda t: 0.25
fast_arr = lambda t: qt.poisson_random_measure(t, fast_rate, 0.5)

ser_t = lambda t: t + 0.2

### Define two Agents
- first with slow arr time
- second with fast arr time: 10 folds faster

In [3]:
def ag_slow(label='Slow'):
    return qt.Agent(
        agent_id = ('slow'),
        arrival_f = slow_arr,
        label= label
)

def ag_fast(label='Fast'):
    return qt.Agent(
        agent_id = ('fast'),
        arrival_f = fast_arr,
        label = label
)

### Setup the network

In [4]:
q_classes = { 1: qt.QueueServer, 2: qt.QueueServer }
q_args = {
    1: {
        'num_server': 1,
        'service_f': ser_t,
        'AgentFactory': ag_slow,
        'collect_data': True
    },
    2: {
        'num_server': 1,
        'service_f': ser_t,
        'AgentFactory': ag_fast,
        'collect_data': True
    }
}

# one starting point, two queues of Agents going into 3 - 4 and exit
adj_list = {0: [1, 2], 1: {3}, 2: {3}, 3: {4}}
edge_list = {0: {1: 1}, 0: {2: 2}}

### Instantiate graph and network

In [5]:
g = qt.adjacency2graph(
    adjacency=adj_list,
    edge_type=edge_list
)

qn = qt.QueueNetwork(
    g=g,
    q_classes=q_classes,
    q_args=q_args
)

### Simulate

In [6]:
qn.start_collecting_data()
qn.initialize(edge_type=[1, 2])
qn.simulate(n=100)  # simulate 100 events
data_out = qn.get_agent_data(return_header=True)

In [7]:
print(data_out[0])

{'slow': array([[0.17663455, 0.17663455, 0.37663455, 0.        , 1.        ,
        3.        ],
       [0.37663455, 0.37663455, 0.57663455, 0.        , 1.        ,
        4.        ],
       [0.57663455, 0.        , 0.        , 0.        , 0.        ,
        5.        ],
       [1.03972075, 1.11195093, 1.31195093, 1.        , 2.        ,
        3.        ],
       [1.31195093, 1.31195093, 1.51195093, 0.        , 1.        ,
        4.        ],
       [1.51195093, 0.        , 0.        , 0.        , 0.        ,
        5.        ],
       [1.81500413, 1.81500413, 0.        , 0.        , 1.        ,
        0.        ],
       [1.89271579, 0.        , 0.        , 1.        , 2.        ,
        0.        ],
       [1.93025887, 1.93025887, 0.        , 0.        , 1.        ,
        2.        ],
       [1.97844102, 2.21500413, 2.41500413, 2.        , 3.        ,
        0.        ],
       [2.01500413, 2.13025887, 2.13025887, 1.        , 2.        ,
        2.        ],
       [2.13

### Go through the data and select fast and slow agents

In [8]:
slow = list(map(lambda x: list(x), data_out[0]['slow']))
fast = list(map(lambda x: list(x), data_out[0]['fast']))

columns = (data_out[1]).split(',')
columns

['arrival', 'service', 'departure', 'num_queued', 'num_total', 'q_id']

### Convert to dataframes...

In [9]:
slow_df = pd.DataFrame(slow, columns=columns)
fast_df = pd.DataFrame(fast, columns=columns)

fast_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   arrival     29 non-null     float64
 1   service     29 non-null     float64
 2   departure   29 non-null     float64
 3   num_queued  29 non-null     float64
 4   num_total   29 non-null     float64
 5   q_id        29 non-null     float64
dtypes: float64(6)
memory usage: 1.5 KB


In [10]:
slow_df['request'] = 'slow'
fast_df['request'] = 'fast'

### ...merge into one and add one column with the time spent in the network. Set to Nan if negative

 Time may be negative for not enough time for this agent to receive service   
 Remove NaN times

In [11]:
all_df = pd.concat([slow_df, fast_df], sort=False)

all_df['times'] = all_df['departure'].sub(all_df['arrival'])

all_df['times'] = all_df['times'].apply(lambda x: x if x > 0 else pd.NA)
all_df.dropna()

,arrival,service,departure,num_queued,num_total,q_id,request,times
0,0.176635,0.176635,0.376635,0.0,1.0,3.0,slow,0.2
1,0.376635,0.376635,0.576635,0.0,1.0,4.0,slow,0.2
3,1.039721,1.111951,1.311951,1.0,2.0,3.0,slow,0.27223
4,1.311951,1.311951,1.511951,0.0,1.0,4.0,slow,0.2
9,1.978441,2.215004,2.415004,2.0,3.0,0.0,slow,0.436563
10,2.015004,2.130259,2.130259,1.0,2.0,2.0,slow,0.115255
11,2.130259,2.130259,2.330259,0.0,1.0,4.0,slow,0.2
12,2.215004,2.330259,2.330259,1.0,2.0,2.0,slow,0.115255
14,2.330259,2.330259,2.530259,0.0,1.0,4.0,slow,0.2
15,2.414454,2.414454,2.614454,0.0,1.0,3.0,slow,0.2


### Store the dataframe into excel file

In [12]:
all_df.to_excel('two_node_md_one.xlsx')